<a href="https://colab.research.google.com/github/S48avio/GOOGLE-ADK/blob/main/Weather_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Step 0: Setup and Installation
# Install ADK and LiteLLM for multi-model support

!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 19.3 MB/s eta 0:00:00
Installation complete.


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Libraries imported.


In [3]:
# @title Configure API Keys (Replace with your actual keys!)

# --- IMPORTANT: Replace placeholders with your real API keys ---

# Gemini API Key (Get from Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "AIzaSyDk2WIsmksssNsBejjjBHdE2YnRYbj-4Rlb3w" # <--- REPLACE



# --- Verify Keys (Optional Check) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **Security Note:** It's best practice to manage API keys securely (e.g., using Colab Secrets or environment variables) rather than hardcoding them directly in the notebook. Replace the placeholder strings above.

API Keys Set:
Google API Key set: Yes


In [4]:
# --- Define Model Constants for easier use ---

# More supported models can be referenced here: https://ai.google.dev/gemini-api/docs/models#model-variations
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"


print("\nEnvironment configured.")


Environment configured.


My weather agent that uses Open-Meteo Api for weather information

In [19]:
import requests

# --- API Endpoints ---
# Step 1: Geocoding API (converts city name to coordinates)
GEOCODING_URL = "https://geocoding-api.open-meteo.com/v1/search"
# Step 2: Forecast API (gets weather data using coordinates)
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"

def get_weather(city: str) -> dict:
    """Retrieves the hourly temperature forecast for a specified city for 1 day.

    This function first converts the city name to coordinates (Geocoding) and
    then fetches the hourly temperature data from the Open-Meteo API.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information or an error message.
    """
    print(f"--- Tool: get_weather called for city: {city} ---")

    try:
        # 1. Geocoding: Convert City Name to Coordinates
        geo_params = {'name': city, 'count': 1, 'language': 'en', 'format': 'json'}
        geo_response = requests.get(GEOCODING_URL, params=geo_params)
        geo_response.raise_for_status()
        geo_data = geo_response.json()

        if not geo_data.get('results'):
            return {"status": "error", "error_message": f"Sorry, could not find coordinates for '{city}'."}

        # Use the first result (usually the most popular city)
        location = geo_data['results'][0]
        latitude = location['latitude']
        longitude = location['longitude']

        found_city = location['name']
        country = location['country']

        # 2. Forecast: Get Hourly Temperature Data
        forecast_params = {
            'latitude': latitude,
            'longitude': longitude,
            'hourly': 'temperature_2m',
            'forecast_days': 1
        }
        forecast_response = requests.get(FORECAST_URL, params=forecast_params)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # 3. Process and Compile Report
        times = forecast_data['hourly']['time']
        temperatures = forecast_data['hourly']['temperature_2m']

        # Find the min/max for the 24-hour period
        min_temp = min(temperatures)
        max_temp = max(temperatures)

        # Create a detailed report (you can customize the output format)
        report = (
            f"Hourly Temperature Forecast for {found_city}, {country} "
            f"(Lat: {latitude:.2f}, Lon: {longitude:.2f}) for the next 24 hours:\n"
            f"Min Temperature: {min_temp}°C\n"
            f"Max Temperature: {max_temp}°C\n"
            f"Hourly Data (Time: Temp):\n"
        )
        # Add a few key hourly points for brevity
        for i in range(0, len(times), 4): # Sample every 4 hours
             report += f"  {times[i].split('T')[1].split(':')[0]}h: {temperatures[i]}°C\n"


        return {
            "status": "success",
            "city": found_city,
            "report": report,
            "min_temperature_celsius": min_temp,
            "max_temperature_celsius": max_temp,
            "hourly_data": list(zip(times, temperatures)) # Provides all raw data
        }

    except requests.exceptions.HTTPError as e:
        return {"status": "error", "error_message": f"HTTP Error: Could not fetch data. ({e})"}

    except requests.exceptions.RequestException:
        return {"status": "error", "error_message": "Connection Error: Could not connect to the API server."}

    except Exception as e:
        return {"status": "error", "error_message": f"An unexpected error occurred: {e}"}

# Example tool usage
print(get_weather("London"))
print(get_weather("Kochi"))
print(get_weather("Nowhereville"))

--- Tool: get_weather called for city: London ---
{'status': 'success', 'city': 'London', 'report': 'Hourly Temperature Forecast for London, United Kingdom (Lat: 51.51, Lon: -0.13) for the next 24 hours:\nMin Temperature: 4.1°C\nMax Temperature: 8.8°C\nHourly Data (Time: Temp):\n  00h: 7.1°C\n  04h: 5.8°C\n  08h: 5.1°C\n  12h: 7.8°C\n  16h: 8.3°C\n  20h: 5.7°C\n', 'min_temperature_celsius': 4.1, 'max_temperature_celsius': 8.8, 'hourly_data': [('2025-11-17T00:00', 7.1), ('2025-11-17T01:00', 6.4), ('2025-11-17T02:00', 5.8), ('2025-11-17T03:00', 5.9), ('2025-11-17T04:00', 5.8), ('2025-11-17T05:00', 5.7), ('2025-11-17T06:00', 5.5), ('2025-11-17T07:00', 5.3), ('2025-11-17T08:00', 5.1), ('2025-11-17T09:00', 5.4), ('2025-11-17T10:00', 6.1), ('2025-11-17T11:00', 7.0), ('2025-11-17T12:00', 7.8), ('2025-11-17T13:00', 8.6), ('2025-11-17T14:00', 8.8), ('2025-11-17T15:00', 8.7), ('2025-11-17T16:00', 8.3), ('2025-11-17T17:00', 7.6), ('2025-11-17T18:00', 6.9), ('2025-11-17T19:00', 6.3), ('2025-11-17T

In [22]:
# @title Define the Weather Agent
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Starting with Gemini

weather_agent = Agent(
    name="weather_agent",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'weather_agent' created using model 'gemini-2.0-flash'.


In [23]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'
Runner created for agent 'weather_agent'.


In [25]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [26]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("How about Paris?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # Expecting the tool's error message

    await call_agent_async("Tell me the weather in New York",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")


>>> User Query: What is the weather like in London?


--- Tool: get_weather called for city: London ---
<<< Agent Response: Here is the weather forecast for London: The temperature will range from a minimum of 4.1°C to a maximum of 8.8°C.

>>> User Query: How about Paris?


--- Tool: get_weather called for city: Paris ---
<<< Agent Response: The weather in Paris will range from a minimum of 4.6°C to a maximum of 9.4°C.

>>> User Query: Tell me the weather in New York


--- Tool: get_weather called for city: New York ---
<<< Agent Response: The weather in New York will range from a minimum of 1.2°C to a maximum of 5.9°C.
